# Analysis of exogenous plant status
Routines to evaluate the status of exogenously given power plants and storage units

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Define data sets and global variables
Use the following data sets:
* maximum installed capacities for exogenously given power plants
* maximum power output for exogenously given plants per year &rarr; indicates installed power for that particular year
* capacities for exogenously given storage units
* maximum installed capacities for RES units
* RES generation time series

Output files comprise
* installed capacity for backup generation
* installed storage capacity
* installed VRES capacities
* annual VRES generation

In [ ]:
# inputs and outputs
path_folder = "./model_inputs/pommesinvest/"
file_name_transformers = "transformers_exogenous.csv"
file_name_transformers_max = "transformers_exogenous_max_ts.csv"
file_name_storages = "storages_el_exogenous.csv"
file_name_res_capacities = "sources_renewables_investment_model.csv"
file_name_res_generation = "sources_renewables_ts_hourly.csv"

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file_names = {
    "backup": "installed_capacity_backup_generation",
    "storages": "installed_storage_capacity",
    "res_capacities": "installed_capacity_res",
    "res_generation": "annual_generation_res"
}

# Plotting and renaming
FUELS = {
    "biomass": "#15b01a",
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "natgas": "#ffd966",
    "hydrogen": "#6fa8dc",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
    "waste": "#c04e01",
    "oil": "#aaa662",
}

STORAGES = {
    "DE_storage_el_PHS": "#0c2aac",
}

RES_NAMES = {
    "DE_source_solarPV":  "solar PV",
    "DE_source_windoffshore": "wind offshore",
    "DE_source_windonshore": "wind onshore",
    "DE_source_biomassEEG": "biomass",
    "DE_source_ROR": "run of river",
    "DE_source_landfillgas": "landfillgas",
    "DE_source_geothermal": "geothermal",
    "DE_source_minegas": "minegas", 
    "DE_source_larga": "sewage gas",
}

RES_COLORS = {
    "solar PV": "#fcb001",
    "wind onshore": "#82cafc",
    "wind offshore": "#0504aa",
    "biomass": "#15b01a",
    "run of river": "#c79fef",
    "other RES": "#757575",
    # "landfillgas": "#06c2ac",
    # "geothermal": "#ff474c",
    # "minegas": "#650021",
    # "sewage gas": "#ad8150", 
}

plt.rcParams.update({'font.size': 14})

## Rearrange and plot data for transformers
* Combine time series and maximum installed capacities to obtain capacities per year
* Groupby fuel and year
* Draw and show plot

In [ ]:
transformers_capacities = pd.read_csv(f"{path_folder}{file_name_transformers}", index_col=0)["capacity"]
transformers_max_ts = pd.read_csv(f"{path_folder}{file_name_transformers_max}", index_col=0)

# Combine information to derive installed capacities by year
transformers_capacity_ts = transformers_max_ts.mul(transformers_capacities)

# Do some reformatting
transformers_capacity_ts.index = transformers_capacity_ts.index.str[:4]
transformers_capacity_ts_transposed = transformers_capacity_ts.T
transformers_capacity_ts_transposed.index = transformers_capacity_ts_transposed.index.str.split("_", expand=True)
transformers_capacity_ts_transposed.index.names = [
    "country", "component_name", "fuel", "type", "cluster", "cluster_no"
]
transformers_capacity_ts_transposed.reset_index(inplace=True)
transformers_capacity_ts_transposed = transformers_capacity_ts_transposed.loc[
    transformers_capacity_ts_transposed["country"] == "DE"
]

# groupby fuel
capacity_by_fuel_year =  transformers_capacity_ts_transposed.groupby("fuel").sum().T

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
capacity_by_fuel_year = capacity_by_fuel_year[[col for col in FUELS.keys()]]
capacity_by_fuel_year.to_csv(f"{path_data_out}{output_file_names['backup']}.csv")
_ = capacity_by_fuel_year.plot(kind="bar", stacked=True, ax=ax, color=FUELS)
_ = plt.xlabel("year")
_ = plt.ylabel("installed capacity in MW")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['backup']}.png", dpi=300)
plt.show()
plt.close()

## Rearrange and plot data for storages
* Read in data
* Reshape and repeat since installed values are kept constant

In [ ]:
storages = pd.read_csv(f"{path_folder}{file_name_storages}", index_col=0)
storages = storages.loc[
    storages["country"] == "DE", 
    ["capacity_pump", "capacity_turbine", "nominal_storable_energy"]
]
storages.rename(
    columns={"capacity_pump": "inflow_power", "capacity_turbine": "outflow_power"},
    inplace=True
)

In [ ]:
storages = pd.DataFrame(
    index=range(2020, 2051), 
    columns=storages["inflow_power"].index, 
    data={
        storages["inflow_power"].index[0]: storages["inflow_power"].values[0],
    }
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
storages = storages[[col for col in STORAGES.keys()]]
storages.to_csv(f"{path_data_out}{output_file_names['storages']}.csv")
_ = storages.plot(kind="bar", stacked=True, ax=ax, color=STORAGES)
_ = plt.xlabel("year")
_ = plt.ylabel("installed capacity in MW")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['storages']}.png", dpi=300)
plt.show()
plt.close()

## Rearrange and plot data for renewables
* Read in installed capacities for 2020 and generation patterns
* Groupby year, calculate and plot annual installed capacities
* Groupby year, calculate and plot annual overall generation

In [ ]:
res_capacities_2020 = pd.read_csv(f"{path_folder}{file_name_res_capacities}", index_col=0)
res_generation = pd.read_csv(f"{path_folder}{file_name_res_generation}", index_col=0)
res_capacities_2020 = res_capacities_2020["capacity"]

In [ ]:
# Groupby year and use maximum and rescale 2020 to a maximum value of 1 to isolate installed capacities
grouped_res = res_generation.groupby(res_generation.index.str[:4])
grouped_res_max = grouped_res.max()
grouped_res_max = grouped_res_max.div(grouped_res_max.loc["2020"])
installed_res_capacities = grouped_res_max.mul(res_capacities_2020).round(0)

In [ ]:
installed_res_capacities.rename(columns=RES_NAMES, inplace=True)
installed_res_capacities["other RES"] = (
    installed_res_capacities["geothermal"] 
    + installed_res_capacities["landfillgas"]
    + installed_res_capacities["minegas"]
    + installed_res_capacities["sewage gas"]
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
installed_res_capacities = installed_res_capacities[[col for col in RES_COLORS]]
installed_res_capacities.to_csv(f"{path_data_out}{output_file_names['res_capacities']}.csv")
_ = installed_res_capacities.plot(kind="bar", stacked=True, ax=ax, color=RES_COLORS)
_ = plt.xlabel("year")
_ = plt.ylabel("installed capacity in MW")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['res_capacities']}.png", dpi=300)
plt.show()
plt.close()

In [ ]:
# Use sum as aggregation rule for generation
grouped_res_sum = grouped_res.sum()
annual_res_generation_in_GWh = grouped_res_sum.mul(res_capacities_2020).round(0).div(1000)

In [ ]:
annual_res_generation_in_GWh.rename(columns=RES_NAMES, inplace=True)
annual_res_generation_in_GWh["other RES"] = (
    annual_res_generation_in_GWh["geothermal"] 
    + annual_res_generation_in_GWh["landfillgas"]
    + annual_res_generation_in_GWh["minegas"]
    + annual_res_generation_in_GWh["sewage gas"]
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
annual_res_generation_in_GWh = annual_res_generation_in_GWh[[col for col in RES_COLORS]]
annual_res_generation_in_GWh.to_csv(f"{path_data_out}{output_file_names['res_generation']}.csv")
_ = annual_res_generation_in_GWh.plot(kind="bar", stacked=True, ax=ax, color=RES_COLORS)
_ = plt.xlabel("year")
_ = plt.ylabel("annual electricity generation in MWh")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['res_generation']}.png", dpi=300)
plt.show()
plt.close()